In [1]:
!pip3 install kfp --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.7/304.7 kB 3.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for kfp: filename=kfp-1.8.17-py3-none-any.whl size=426971 sha256=3d7cec5ff322cad325a0b9f33dda1166407e32f23b36a8de11c12eb74dffb74a
  Stored in directory: /home/jupyter/.cache/pip/wheels/cc/43/c4/288dbaf053698d442f56b20a1e122092b99ac6e7c987b54a69
Successfully built kfp
  Attempting uninstall: kfp
    Found existing installation: kfp 1.8.16
    Uninstalling kfp-1.8.16:
      Successfully uninstalled kfp-1.8.16


In [3]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google.client import AIPlatformClient

In [4]:
PROJECT_ID = 'digital-proton-368805'
REGION = 'us-central1'
BUCKET_NAME = "gs://vertext-ai-piepline"

In [5]:
PIPELINE_ROOT = "{}/pipeline_root1".format(BUCKET_NAME)

In [6]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [7]:
@component(output_component_file="add.yaml", base_image="python:3.9") 
def add(a: float, b: float) -> float:
    return a + b

In [8]:
@component(output_component_file="multiply.yaml", base_image="python:3.9") 
def multiply(a: float, b: float) -> float:
    return a * b

In [9]:
@dsl.pipeline(name="my-first-pipeline1", pipeline_root=PIPELINE_ROOT)
def add_mul_pipeline(a: float= 3, b: float=5):
    result = add(a, b)
    multiply(result.output, b)

In [11]:
compiler.Compiler().compile(pipeline_func=add_mul_pipeline, package_path="p1.json")

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [12]:
from kfp.v2.google.client import AIPlatformClient
api_client = AIPlatformClient(project_id=PROJECT_ID, region=REGION)

/opt/conda/lib/python3.7/site-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v2.0.0. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


In [ ]:
response = api_client.create_run_from_job_spec(job_spec_path="p1.json")